#### Daily Data Pull

In [1]:
# imports
import pandas as pd
import numpy as np
import requests
import time
import datetime
from nltk.tokenize import RegexpTokenizer

import os

In [2]:
# set display options 
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', None)

In [3]:
# define function to return minimum postedDate as string
def get_min_post_date(filename):
    # read in file of downloaded contract opportunities
    df = pd.read_csv(filename)
    # get min posted date
    min_date = pd.to_datetime(df['postedDate']).min()
    return min_date.strftime('%m/%d/%Y')

In [4]:
# define function to return maximum postedDate as string
def get_max_post_date(filename):
    # read in file of downloaded contract opportunities
    df = pd.read_csv(filename)
    # get min posted date
    max_date = pd.to_datetime(df['postedDate']).max()
    return max_date.strftime('%m/%d/%Y')

In [5]:
# define function to return today's date as string
def today():
    now = datetime.datetime.now()
    return now.strftime('%m/%d/%Y')

In [6]:
# pull contract information from api.sam.gov

# import api key from environment 
api_key = os.environ.get('beta_sam_gov_key')

# set base url
url = 'https://api.sam.gov/prod/opportunities/v1/search'

# create empty list to store results
result = []
# initialize counter
count = 0
# downloaded contract opportunities - file name
# file = './data/combined.csv'

# set postedTo date to today's date by calling today() function
postedTo = today()

# set posted from date
postedFrom = get_max_post_date('./data/combined.csv')

# for loop to pull contracts
for i in range(2):
    count += 1
    
    # do a get request
    req = requests.get(url,
                      params={
                          'api_key': api_key,
                          'postedFrom': postedFrom,
                          'postedTo': postedTo,
                          'limit': 1000,
                          'offset': (count-1) * 1000
                      })
    
    # add response to result list
    result.append(req)
    
    now = datetime.datetime.now()
    print('Time:', now.strftime("%Y-%m-%d %H:%M:%S"))
    time.sleep(5)
    
    
    
# source for datetime - https://www.w3resource.com/python-exercises/python-basic-exercise-3.php

Time: 2020-06-28 20:09:49
Time: 2020-06-28 20:10:13


In [7]:
# unpack list of json objects from response data
ops = []
for item in result:
    print(item.headers)
    ops.append(item.json())

{'Age': '23', 'Content-Type': 'application/hal+json', 'Date': 'Mon, 29 Jun 2020 00:09:49 GMT', 'Server': 'openresty', 'Vary': 'Origin, Access-Control-Request-Method, Access-Control-Request-Headers', 'Via': 'http/1.1 api-umbrella (ApacheTrafficServer [cMsSf ])', 'X-Cache': 'MISS', 'X-Forwarded-For': '74.96.156.35, 10.177.16.72, 10.177.52.86, 10.177.52.86', 'Connection': 'keep-alive', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload', 'Set-Cookie': 'citrix_ns_id=tJc98jUBMWiVzNAePWNcGjVXmt40001; Domain=.sam.gov; Path=/; Secure; HttpOnly', 'Cache-Control': 'private', 'Content-Encoding': 'gzip', 'Transfer-Encoding': 'chunked'}
{'Age': '19', 'Content-Type': 'application/hal+json', 'Date': 'Mon, 29 Jun 2020 00:10:13 GMT', 'Server': 'openresty', 'Vary': 'Origin, Access-Control-Request-Method, Access-Control-Request-Headers', 'Via': 'http/1.1 api-umbrella (ApacheTrafficServer [cMsSf ])', 'X-Cache': 'MISS', 'X-Forwarded-For': '74.96.156.35, 10.177.16.72, 10.177.54.205, 

In [8]:
# parse json objects
ls_data = []
for i in range(len(ops)):
    print(ops[i].keys())
    df = pd.DataFrame(ops[i]['opportunitiesData'])
    ls_data.append(df)
    data = pd.concat(ls_data)

dict_keys(['totalRecords', 'limit', 'offset', 'opportunitiesData', 'links'])
dict_keys(['totalRecords', 'limit', 'offset', 'opportunitiesData', 'links'])


In [9]:
# check how many new data points were downloaded
data.shape

(1576, 27)

In [10]:
data.tail(10)

,noticeId,title,solicitationNumber,department,subTier,office,postedDate,type,baseType,archiveType,archiveDate,typeOfSetAsideDescription,typeOfSetAside,responseDeadLine,naicsCode,classificationCode,active,award,pointOfContact,description,organizationType,officeAddress,placeOfPerformance,additionalInfoLink,uiLink,links,resourceLinks
566,028cf5870db945b99d937e4ded2a9d0a,J045--Chilled Water Coil replacement,36C24620Q0495_3,"VETERANS AFFAIRS, DEPARTMENT OF","VETERANS AFFAIRS, DEPARTMENT OF",246-NETWORK CONTRACTING OFFICE 6 (36C246),2020-06-26,Presolicitation,Sources Sought,autocustom,2020-08-26,Service-Disabled Veteran-Owned Small Business (SDVOSB) Set-Aside (FAR 19.14),SDVOSBC,2020-06-26T15:00:00-04:00,238220,J045,Yes,None,"[{'fax': None, 'type': 'primary', 'email': 'breon.campbell@va.gov', 'phone': None, 'title': None, 'fullName': 'Breon Campbell breon.campbell@va.gov'}]",https://api.sam.gov/prod/opportunities/v1/noticedesc?noticeid=028cf5870db945b99d937e4ded2a9d0a,OFFICE,"{'zipcode': '23667', 'city': 'HAMPTON', 'countryCode': 'USA', 'state': 'VA'}","{'city': {'code': '67000', 'name': 'Richmond'}, 'state': {'code': 'VA', 'name': 'Virginia'}, 'zip': '23249', 'country': {'code': 'USA', 'name': 'UNITED STATES'}}",None,https://beta.sam.gov/opp/028cf5870db945b99d937e4ded2a9d0a/view,"[{'rel': 'self', 'href': 'https://api.sam.gov/prod/opportunities/v1/search?noticeid=028cf5870db945b99d937e4ded2a9d0a&limit=1'}]",[https://beta.sam.gov/api/prod/opps/v3/opportunities/resources/files/9bbde7388f5746cfb066c777f5607109/download?api_key=null&token=]
567,0277e56e75db49a89ac6b70cde4ec7e8,Internal Combustion Lift Truck - Forklifts,W912L920R0024,DEPT OF DEFENSE,DEPT OF THE ARMY,W7M7 USPFO ACTIVITY IN ARNG,2020-06-26,Combined Synopsis/Solicitation,Combined Synopsis/Solicitation,auto15,2020-07-25,Total Small Business Set-Aside (FAR 19.5),SBA,2020-07-10T10:00:00-04:00,333924,2420,Yes,None,"[{'fax': '', 'type': 'primary', 'email': 'brenda.j.simmons6.civ@mail.mil', 'phone': '3172473154', 'title': None, 'fullName': 'Brenda J. Simmons'}]",https://api.sam.gov/prod/opportunities/v1/noticedesc?noticeid=0277e56e75db49a89ac6b70cde4ec7e8,OFFICE,"{'zipcode': '46241-4839', 'city': 'INDIANAPOLIS', 'countryCode': 'USA', 'state': 'IN'}","{'city': {'code': '36000', 'name': 'Indianapolis'}, 'state': {'code': 'IN', 'name': 'Indiana'}, 'zip': '46241', 'country': {'code': 'USA', 'name': 'UNITED STATES'}}",None,https://beta.sam.gov/opp/0277e56e75db49a89ac6b70cde4ec7e8/view,"[{'rel': 'self', 'href': 'https://api.sam.gov/prod/opportunities/v1/search?noticeid=0277e56e75db49a89ac6b70cde4ec7e8&limit=1'}]","[https://beta.sam.gov/api/prod/opps/v3/opportunities/resources/files/0cc756763a8c4791912ab16e0c8ac64a/download?api_key=null&token=, https://beta.sam.gov/api/prod/opps/v3/opportunities/resources/files/a219ba96b57d473eafcebd1984bbd6ed/download?api_key=null&token=]"
568,027229651ce3463fba2a625c5e27da4f,B--North American Bat Monitoring Program (NABat) Surv,140P6220Q0015,"INTERIOR, DEPARTMENT OF THE",NATIONAL PARK SERVICE,MWR MISSOURI MABO(62000),2020-06-26,Solicitation,Solicitation,auto15,2020-07-16,Total Small Business Set-Aside (FAR 19.5),SBA,2020-07-01,541620,B,Yes,{'awardee': {'location': {}}},"[{'fax': None, 'type': 'primary', 'email': 'Kathryn_Logsdon@nps.gov', 'phone': None, 'title': None, 'fullName': 'Logsdon, Kathryn'}]",https://api.sam.gov/prod/opportunities/v1/noticedesc?noticeid=027229651ce3463fba2a625c5e27da4f,OFFICE,"{'zipcode': '63102', 'city': 'SAINT LOUIS', 'countryCode': 'USA', 'state': 'MO'}","{'streetAddress': 'Ozark National Scenic Riverways, NABat Grid Cell Numbers , 5999, 10095, 1775, Van Buren, MO', 'zip': '63965-9603', 'country': {'code': 'USA', 'name': 'UNITED STATES'}}",None,https://beta.sam.gov/opp/027229651ce3463fba2a625c5e27da4f/view,"[{'rel': 'self', 'href': 'https://api.sam.gov/prod/opportunities/v1/search?noticeid=027229651ce3463fba2a625c5e27da4f&limit=1'}]","[https://beta.sam.gov/api/prod/opps/v3/opportunities/resources/files/7281abae448743f19712

In [11]:
# check if there are duplicate documents in the newly pulled data
data.duplicated('noticeId').sum()

0

In [12]:
# if duplicates found, drop those
data.drop_duplicates('noticeId', inplace=True)

In [12]:
# save data into a new file
data.to_csv('./data/6_28_pull.csv', index=False)

**Note:** Because in Notebook 008 the combined.csv dataframe gets two additional columns -  'cleaned_titles' and 'date_posted', before concatenating the new pull with the combined.csv, we need a function that transforms the newly acquired data into the same shape.

In [13]:
# define function to tokenize a column 
# modified to return title cased titles

def tokenizer_function(column):
    """
    Takes in a text column
        tokenizes the text in each row
        using pattern [[a-zA-Z]\w+]
        which matches every lowercase and upperase character between a-z that are word characters
    Returns list of strings
    """
    
    # instantiate empty list of tokenized text
    texts = []
    
    # define tokenizer pattern
    pattern = '[a-zA-Z]\w+'
    # instantiate tokenizer
    tokenizer = RegexpTokenizer(pattern=pattern)
    
    # create for loop to tokenize each row and add the list of tokens to texts
    for text in column:
        tokens = tokenizer.tokenize(text)
        
        # transform tokens into lower case strings
        tokens = [token.title() for token in tokens]
        texts.append(' '.join(tokens))
    return texts

In [14]:
def transform_new_pull(new_data):
    """function
       accepts new_data
       adds a tokenized title column to new_data
       adds a column of posted date in datetime format to new_data
       returns transformed new_data as dataframe"""
    # create tokenized title column
    new_data['cleaned_titles'] = tokenizer_function(new_data['title'])
    
    # create date-posted column in datetime format
    new_data['date_posted'] = pd.to_datetime(new_data['postedDate'])
    
    return new_data

In [15]:
# transform new dataframe
data = transform_new_pull(data)

In [16]:
# save transformed dataframe into a csv file
data.to_csv('./data/6_28_pull_transformed.csv', index=False)

In [17]:
# read in both the combined dataframe and the new-tranformed dataframe
df1 = pd.read_csv('./data/combined.csv')
df2 = pd.read_csv('./data/6_28_pull_transformed.csv')

In [18]:
# concatenate the new dataframe with the combined dataframe
df = pd.concat([df1, df2])

In [19]:
# check new shape
df.shape

(66614, 29)

In [20]:
# overwrite the combined csv file with the newly combined dataframe
df.to_csv('./data/combined.csv', index=False)